# Contract Analysis
- We apply the neural network with pre-trained weights on norms contained in contracts.

In [1]:
import pandas as pd
from pandas import DataFrame

results_df = pd.read_csv('../result.tsv', sep='\t')
results_df['norm1'] = results_df['norm1'].astype(str)
results_df['len_norm1'] = results_df['norm1'].apply(lambda row: len(row.split()))
results_df['norm2'] = results_df['norm2'].astype(str)
results_df['len_norm2'] = results_df['norm2'].apply(lambda row: len(row.split()))
len_df = results_df[['len_norm1', 'len_norm2']]
display(results_df.columns)
len_df.describe(percentiles=[.25, .5, .75, .99])

Index(['norm1', 'norm2', 'conflict', 'majority_class', 'entailment_prob',
       'contradiction_prob', 'neutral_prob', 'len_norm1', 'len_norm2'],
      dtype='object')

,len_norm1,len_norm2
count,13742.000000,13742.000000
mean,34.560690,34.607335
std,34.305647,33.738959
min,1.000000,1.000000
25%,15.000000,16.000000
50%,26.000000,26.000000
75%,43.000000,43.000000
99%,167.000000,163.000000
max,427.000000,473.000000


In [6]:
conflicts_df = results_df[results_df['conflict']]
conflicts_df['majority_class'].describe()

count         228
unique          3
top       neutral
freq          119
Name: majority_class, dtype: object

In [10]:
entail_conflict_df = conflicts_df[conflicts_df['majority_class'] == 'entailment']
entail_conflict_df.to_csv('entail_conflicts.tsv', sep='\t')
entail_conflict_df.describe()

,entailment_prob,contradiction_prob,neutral_prob,len_norm1,len_norm2
count,63.000000,63.000000,63.000000,63.000000,63.000000
mean,-0.347410,-3.229181,-1.872524,23.952381,18.111111
std,0.207560,1.480649,0.738786,11.141941,9.021678
min,-0.959303,-6.606812,-4.046466,1.000000,3.000000
25%,-0.446924,-3.983119,-2.235459,16.000000,12.500000
50%,-0.355829,-3.171389,-1.687096,22.000000,15.000000
75%,-0.196351,-2.111051,-1.334587,28.500000,24.500000
max,-0.024838,-0.807238,-0.900276,75.000000,46.000000


In [11]:
con_conflict_df = conflicts_df[conflicts_df['majority_class'] == 'contradiction']
con_conflict_df.to_csv('contradiction_conflicts.tsv', sep='\t')
con_conflict_df.describe()


,entailment_prob,contradiction_prob,neutral_prob,len_norm1,len_norm2
count,46.000000,46.000000,46.000000,46.000000,46.000000
mean,-3.003177,-0.270916,-2.826320,19.000000,14.478261
std,1.555031,0.231025,1.254055,7.192589,5.935727
min,-6.887687,-0.769118,-5.898938,7.000000,5.000000
25%,-4.005798,-0.408288,-3.581236,16.000000,9.250000
50%,-2.780901,-0.230343,-2.383924,17.000000,17.000000
75%,-1.766081,-0.060858,-1.959570,22.000000,18.750000
max,-0.823343,-0.005169,-0.768717,44.000000,25.000000
